In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'train'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0,1,COMPLETE,2020-01-01 04:34:23.549454+00:00,2,0 days 00:36:43.505333,2203,14260,1,0,0,0,0,1,0,0
1,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0,4,COMPLETE,2020-01-01 14:30:27.423999+00:00,5,0 days 09:56:03.874545,35763,16463,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 08:16:35.844753+00:00,Jane,1,10,COMPLETE,2020-01-01 08:47:14.772217+00:00,11,0 days 00:30:38.927464,1838,29795,1,0,0,0,0,1,0,0
4,REPAIR,START,2020-01-01 08:47:14.772217+00:00,Karsten,1,13,COMPLETE,2020-01-01 13:27:22.316694+00:00,14,0 days 04:40:07.544477,16807,31634,1,0,1,0,0,1,0,1
5,QUALITY_CONTROL,START,2020-01-01 13:27:22.316694+00:00,Clark,1,16,COMPLETE,2020-01-01 17:21:42.825766+00:00,17,0 days 03:54:20.509072,14060,48442,1,1,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5400,DIAGNOSIS,START,2024-12-26 18:05:59.540931+00:00,Clark,1800,16201,COMPLETE,2024-12-26 19:13:26.892809+00:00,16202,0 days 01:07:27.351878,4047,65159,1,0,0,0,1,0,0,0
5399,QUALITY_CONTROL,START,2024-12-26 19:03:41.622915+00:00,Joe,1799,16198,COMPLETE,2024-12-26 21:39:17.312456+00:00,16199,0 days 02:35:35.689541,9335,68621,1,1,1,1,0,0,1,1
5401,REPAIR,START,2024-12-26 19:13:26.892809+00:00,Clark,1800,16204,COMPLETE,2024-12-27 08:23:46.946760+00:00,16205,0 days 13:10:20.053951,47420,69206,1,0,1,0,2,0,0,0
5402,QUALITY_CONTROL,START,2024-12-27 08:23:46.946760+00:00,Jane,1800,16207,COMPLETE,2024-12-27 11:10:44.009123+00:00,16208,0 days 02:46:57.062363,10017,30226,1,1,1,0,2,1,0,0


In [4]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/artificial/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A = evaluator_A.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:08<00:00, 14.03it/s]


In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('-11.29103954556296487899430175')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(11106.03273482976)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/artificial/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:04<00:00, 14.53it/s]


In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('-11.09089646766912641549289979')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(9804.988686291626)

In [11]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/artificial/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:15<00:00, 13.33it/s]


In [12]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

Decimal('-11.25767775707060618772908362')

In [13]:
np.mean(get_pscores(likelihoods_R_S))

np.float64(11600.358034398065)

In [14]:
drbart_model_R = DRBART(parser_dir = '../../../models/artificial/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_R = evaluator_R.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:32<00:00, 11.85it/s]


In [15]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

Decimal('-11.34352400266938185940497096')

In [16]:
np.mean(get_pscores(likelihoods_R))

np.float64(13239.542188074212)

In [17]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:02<00:00, 14.73it/s]


In [18]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('-11.13809134103246568679812704')

In [19]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(10215.267228666871)

In [20]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:09<00:00, 13.91it/s]


In [21]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('-11.10127085157318913387291713')

In [22]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(9936.679286332079)

In [23]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:24<00:00, 12.46it/s]


In [24]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('-11.05219798646674340129010193')

In [25]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(8397.386798415502)

In [26]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

100%|███████████████████████████████████████████████| 1802/1802 [02:10<00:00, 13.86it/s]


In [27]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('-10.95031050303239743799785854')

In [28]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(8076.855230038752)

In [29]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./artificial_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)

In [30]:
with open('./artificial_dr_bart_evaluation_'+log_name+'.pickle', 'rb') as handle:
    results = pickle.load(handle)